In [ ]:
!pip install scikeras
 

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import ParameterGrid
from scikeras.wrappers import KerasClassifier  

train_dir = r"C:\Users\pc\Desktop\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)
image_size = (224, 224)

X = []
y = []

for category in categories:
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            X.append(np.array(image))
            y.append(category)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

def build_model(learning_rate=0.001):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    model = KerasClassifier(
    model=build_model,
    verbose=1
)

param_grid = {
    'model__learning_rate': [0.001, 0.0001, 0.002, 0.0002],
    'batch_size': [32, 64, 128],
    'epochs': [5, 10, 15, 20]
}
best_score = 0
best_params = {}

print("\nStarting Grid Search with the following parameters:\n")

for params in ParameterGrid(param_grid):
    print(f"Training with: learning_rate={params['model__learning_rate']}, batch_size={params['batch_size']}, epochs={params['epochs']}")
    model.set_params(**params)
    model.fit(X_train, y_train_one_hot)
    score = model.score(X_val, y_val_one_hot)
    print(f"Accuracy: {score:.4f}\n")
    if score > best_score:
        best_score = score
        best_params = params
print("\nBest Parameters:", best_params)
print("Best Accuracy Score:", best_score)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

train_dir = r"C:\Users\CS\Downloads\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)
image_size = (224, 224)

X = []
y = []

for category in categories:
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            X.append(np.array(image))
            y.append(category)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocess_input_vgg16(X_train)
X_val = preprocess_input_vgg16(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

def build_vgg16_model(learning_rate=0.001):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

keras_model_vgg16 = KerasClassifier(
    model=build_vgg16_model,
    verbose=1
)

param_grid_vgg16 = {
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [5, 10]
}

best_score_vgg16 = 0
best_params_vgg16 = {}

print("\nStarting Grid Search for VGG16 with the following parameters:\n")

for params in ParameterGrid(param_grid_vgg16):
    print(f"Training VGG16 with: learning_rate={params['model__learning_rate']}, batch_size={params['batch_size']}, epochs={params['epochs']}")
    keras_model_vgg16.set_params(**params)
    keras_model_vgg16.fit(X_train, y_train_one_hot)
    score = keras_model_vgg16.score(X_val, y_val_one_hot)
    print(f"VGG16 Accuracy: {score:.4f}\n")
    if score > best_score_vgg16:
        best_score_vgg16 = score
        best_params_vgg16 = params

print("\nBest Parameters for VGG16:", best_params_vgg16)
print("Best Accuracy Score for VGG16:", best_score_vgg16)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_inceptionv3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

print("\n--- InceptionV3 Model ---")

# مسار البيانات
train_dir = r"C:\Users\pc\Desktop\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)
image_size = (299, 299) # InceptionV3 يتطلب هذا الحجم

X = []
y = []

for category in categories:
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            X.append(np.array(image))
            y.append(category)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocess_input_inceptionv3(X_train)
X_val = preprocess_input_inceptionv3(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

def build_inceptionv3_model(learning_rate=0.001):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    base_model.trainable = False 
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x) 
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

keras_model_inceptionv3 = KerasClassifier(
    model=build_inceptionv3_model,
    verbose=1
)

param_grid_inceptionv3 = {
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [5, 10]
}

best_score_inceptionv3 = 0
best_params_inceptionv3 = {}

print("\nStarting Grid Search for InceptionV3 with the following parameters:\n")

for params in ParameterGrid(param_grid_inceptionv3):
    print(f"Training InceptionV3 with: learning_rate={params['model__learning_rate']}, batch_size={params['batch_size']}, epochs={params['epochs']}")
    keras_model_inceptionv3.set_params(**params)
    keras_model_inceptionv3.fit(X_train, y_train_one_hot)
    score = keras_model_inceptionv3.score(X_val, y_val_one_hot)
    print(f"InceptionV3 Accuracy: {score:.4f}\n")
    if score > best_score_inceptionv3:
        best_score_inceptionv3 = score
        best_params_inceptionv3 = params

print("\nBest Parameters for InceptionV3:", best_params_inceptionv3)
print("Best Accuracy Score for InceptionV3:", best_score_inceptionv3)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_input_mobilenet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier


train_dir = r"C:\Users\pc\Desktop\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)
image_size = (224, 224)

X = []
y = []

for category in categories:
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            X.append(np.array(image))
            y.append(category)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocess_input_mobilenet(X_train)
X_val = preprocess_input_mobilenet(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

def build_mobilenet_model(learning_rate=0.001):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x) 
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

keras_model_mobilenet = KerasClassifier(
    model=build_mobilenet_model,
    verbose=1
)

param_grid_mobilenet = {
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [5, 10]
}

best_score_mobilenet = 0
best_params_mobilenet = {}

print("\nStarting Grid Search for MobileNet with the following parameters:\n")

for params in ParameterGrid(param_grid_mobilenet):
    print(f"Training MobileNet with: learning_rate={params['model__learning_rate']}, batch_size={params['batch_size']}, epochs={params['epochs']}")
    keras_model_mobilenet.set_params(**params)
    keras_model_mobilenet.fit(X_train, y_train_one_hot)
    score = keras_model_mobilenet.score(X_val, y_val_one_hot)
    print(f"MobileNet Accuracy: {score:.4f}\n")
    if score > best_score_mobilenet:
        best_score_mobilenet = score
        best_params_mobilenet = params

print("\nBest Parameters for MobileNet:", best_params_mobilenet)
print("Best Accuracy Score for MobileNet:", best_score_mobilenet)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier


train_dir = r"C:\Users\pc\Desktop\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)
image_size = (224, 224)

X = []
y = []

for category in categories:
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            X.append(np.array(image))
            y.append(category)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocess_input_resnet50(X_train)
X_val = preprocess_input_resnet50(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)

def build_resnet50_model(learning_rate=0.001):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x) 
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

keras_model_resnet50 = KerasClassifier(
    model=build_resnet50_model,
    verbose=1
)

param_grid_resnet50 = {
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [5, 10]
}

best_score_resnet50 = 0
best_params_resnet50 = {}

print("\nStarting Grid Search for ResNet50 with the following parameters:\n")

for params in ParameterGrid(param_grid_resnet50):
    print(f"Training ResNet50 with: learning_rate={params['model__learning_rate']}, batch_size={params['batch_size']}, epochs={params['epochs']}")
    keras_model_resnet50.set_params(**params)
    keras_model_resnet50.fit(X_train, y_train_one_hot)
    score = keras_model_resnet50.score(X_val, y_val_one_hot)
    print(f"ResNet50 Accuracy: {score:.4f}\n")
    if score > best_score_resnet50:
        best_score_resnet50 = score
        best_params_resnet50 = params

print("\nBest Parameters for ResNet50:", best_params_resnet50)
print("Best Accuracy Score for ResNet50:", best_score_resnet50)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import random
import cv2 


In [ ]:
train_dir = r"C:\Users\CS\Downloads\skin-disease-datasaet\train_set"
categories = os.listdir(train_dir)

In [ ]:
image_size = (224, 224) 
X = []  
y = []  
for idx, category in enumerate(categories):
    category_path = os.path.join(train_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)
        try:
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            X.append(image_array)
            y.append(category) 
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

X = np.array(X)
y = np.array(y)
unique_labels = np.unique(y)  
fig, axes = plt.subplots(2, 4, figsize=(12, 6))  
plt.suptitle('One Image per Label', fontsize=16)
for label, ax in zip(unique_labels, axes.flatten()):
    idx = np.where(y == label)[0][0]  
    img = X[idx]
    ax.axis('off')
    ax.set_title(str(label), fontsize=8)
    ax.imshow(img)
plt.tight_layout()


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

num_classes = len(categories)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224 , 3))
base_model.trainable = False
num_classes = 8
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
EPOCHS = 20
BATCH_SIZE = 32
history = model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),
                   epochs = EPOCHS, batch_size=BATCH_SIZE)
train_loss, train_accuracy = model.evaluate(X_train, y_train_one_hot, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

val_loss, val_accuracy = model.evaluate(X_val, y_val_one_hot, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.tight_layout()
plt.show()
model.save("ResNet50_skin_disease_model.keras")

model = keras.models.load_model("ResNet50_skin_disease_model.keras")

test_path = r"C:\Users\CS\Downloads\skin-disease-datasaet\test_set"
real_label = []
predicted_class = []
le = LabelEncoder()
le.fit(y_train) 
for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))  
        img = np.array([img]) 
        img = preprocess_input(img) 
        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])  
        for real, predicted in zip(real_label[:5], predicted_class[:5]):
        print(f"Real: {real}, Predicted: {predicted}")
        
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
y_pred = model.predict(X_val) 
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_val_encoded, y_pred_classes)  
accuracy = accuracy_score(y_val_encoded, y_pred_classes) 
precision = precision_score(y_val_encoded, y_pred_classes, average='weighted')  
recall = recall_score(y_val_encoded, y_pred_classes, average='weighted')  
f1 = f1_score(y_val_encoded, y_pred_classes, average='weighted') 
y_pred_proba = model.predict(X_val)  
y_pred_proba = y_pred_proba 
auc = roc_auc_score(y_val_encoded, y_pred_proba, multi_class='ovr', average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC curve:", auc)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:

def build_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=20,
    batch_size=64,
    verbose=1
)
train_loss, train_accuracy = model.evaluate(X_train, y_train_one_hot, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

val_loss, val_accuracy = model.evaluate(X_val, y_val_one_hot, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.tight_layout()
plt.show()

model.save("efficientnet_skin_disease_model1.keras")
from tensorflow.keras.models import load_model
model = load_model("efficientnet_skin_disease_model1.keras")

test_path = r"C:\Users\CS\Downloads\skin-disease-datasaet\test_set"
real_label = []
predicted_class = []
le = LabelEncoder()
le.fit(y_train) 
for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))  
        img = np.array([img]) 
        img = preprocess_input(img) 
        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])  
        for real, predicted in zip(real_label[:5], predicted_class[:5]):
        print(f"Real: {real}, Predicted: {predicted}")

from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
y_pred = model.predict(X_val) 
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_val_encoded, y_pred_classes)  
accuracy = accuracy_score(y_val_encoded, y_pred_classes) 
precision = precision_score(y_val_encoded, y_pred_classes, average='weighted')  
recall = recall_score(y_val_encoded, y_pred_classes, average='weighted')  
f1 = f1_score(y_val_encoded, y_pred_classes, average='weighted') 
y_pred_proba = model.predict(X_val)  
y_pred_proba = y_pred_proba 
auc = roc_auc_score(y_val_encoded, y_pred_proba, multi_class='ovr', average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC curve:", auc)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
   
def build_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False 

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=20,
    batch_size=32,
    verbose=1
)

val_loss, val_accuracy = model.evaluate(X_val, y_val_one_hot)
print(f"Validation Accuracy: {val_accuracy:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.tight_layout()
plt.show()
model.save("VGG16_skin_disease_model.keras")

model = keras.models.load_model("VGG16_skin_disease_model.keras")

test_path = r"C:\Users\CS\Downloads\skin-disease-datasaet\test_set"
real_label = []
predicted_class = []
le = LabelEncoder()
le.fit(y_train) 
for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224))  
        img = np.array([img]) 
        img = preprocess_input(img) 
        predictions = model.predict(img)
        real_label.append(folder)
        predicted_class_index = np.argmax(predictions)
        predicted_class.append(le.classes_[predicted_class_index])  
        for real, predicted in zip(real_label[:5], predicted_class[:5]):
        print(f"Real: {real}, Predicted: {predicted}")

from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
y_pred = model.predict(X_val) 
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_val_encoded, y_pred_classes)  
accuracy = accuracy_score(y_val_encoded, y_pred_classes) 
precision = precision_score(y_val_encoded, y_pred_classes, average='weighted')  
recall = recall_score(y_val_encoded, y_pred_classes, average='weighted')  
f1 = f1_score(y_val_encoded, y_pred_classes, average='weighted') 
y_pred_proba = model.predict(X_val)  
y_pred_proba = y_pred_proba 
auc = roc_auc_score(y_val_encoded, y_pred_proba, multi_class='ovr', average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC curve:", auc)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

        


In [ ]:
def build_inceptionv3_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    base_model.trainable = False 
    x = base_model.output
    x = GlobalAveragePooling2D()(x) 
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x) 

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# بناء النموذج
model = build_inceptionv3_model()
model.summary()

history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=20, 
    batch_size=32, 
    verbose=1 
)

print("\nEvaluating InceptionV3 model on validation data...")
val_loss, val_accuracy = model.evaluate(X_val, y_val_one_hot, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy (InceptionV3)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (InceptionV3)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

model_save_path = "InceptionV3_skin_disease_model.keras"
model.save(model_save_path)
loaded_model = keras.models.load_model(model_save_path)

real_label = []
predicted_class = []


for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Warning: Could not read image {file_path}. Skipping.")
                    continue
                img = cv2.resize(img, image_size) 
                img = np.expand_dims(img, axis=0) 
                img = preprocess_input(img)
                predictions = loaded_model.predict(img, verbose=0) 
                real_label.append(folder) 
                predicted_class_index = np.argmax(predictions)
                predicted_class.append(le.classes_[predicted_class_index]) 

            except Exception as e:
                print(f"Error processing image {file_path}: {e}")

print("\nSample predictions from test set:")
for real, predicted in zip(real_label[:5], predicted_class[:5]):
    print(f"Real: {real}, Predicted: {predicted}")


y_pred = loaded_model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1) 

conf_matrix = confusion_matrix(y_val_encoded, y_pred_classes)

accuracy = accuracy_score(y_val_encoded, y_pred_classes)
precision = precision_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0) # zero_division=0 لتجنب الأخطاء
recall = recall_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0)

y_pred_proba = loaded_model.predict(X_val)

auc = roc_auc_score(y_val_encoded, y_pred_proba, multi_class='ovr', average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC Score: {auc:.4f}")

plt.figure(figsize=(10, 8)) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_) 
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (InceptionV3)')
plt.show()


In [ ]:
def build_mobilenet_model():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x) 
    x = Dense(512, activation='relu')(x) 
    predictions = Dense(num_classes, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_mobilenet_model()
model.summary()
history = model.fit(
    X_train, y_train_one_hot,
    validation_data=(X_val, y_val_one_hot),
    epochs=10,
    batch_size=32,
    verbose=1 
)

val_loss, val_accuracy = model.evaluate(X_val, y_val_one_hot, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy (MobileNet)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (MobileNet)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


model_save_path = "MobileNet_skin_disease_model.keras"
model.save(model_save_path)
loaded_model = keras.models.load_model(model_save_path)

print("Starting predictions on test set...")
real_label = []
predicted_class = []

for folder in sorted(os.listdir(test_path)):
    folder_path = os.path.join(test_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Warning: Could not read image {file_path}. Skipping.")
                    continue
                img = cv2.resize(img, image_size) 
                img = np.expand_dims(img, axis=0) 
                img = preprocess_input(img) 
                predictions = loaded_model.predict(img, verbose=0)
                real_label.append(folder)
                predicted_class_index = np.argmax(predictions)
                predicted_class.append(le.classes_[predicted_class_index])
            except Exception as e:
                print(f"Error processing image {file_path}: {e}")

print("\nSample predictions from test set:")
for real, predicted in zip(real_label[:5], predicted_class[:5]):
    print(f"Real: {real}, Predicted: {predicted}")

y_pred = loaded_model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1) 

conf_matrix = confusion_matrix(y_val_encoded, y_pred_classes)

accuracy = accuracy_score(y_val_encoded, y_pred_classes)
precision = precision_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0) 
recall = recall_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_val_encoded, y_pred_classes, average='weighted', zero_division=0)

# ROC AUC Score
y_pred_proba = loaded_model.predict(X_val)
auc = roc_auc_score(y_val_encoded, y_pred_proba, multi_class='ovr', average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC Score: {auc:.4f}")

plt.figure(figsize=(10, 8)) 
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_) 
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (MobileNet)')
plt.show()


In [ ]:
import cv2
import numpy as np
import imgaug.augmenters as iaa
import os
from glob import glob

input_folder = r"C:\Users\CS\Downloads\skin-disease-datasaet\train_set\FU-ringworm"
output_folder = r"C:\Users\CS\Downloads\skin-disease-datasaet\train_set\New folder"

os.makedirs(output_folder, exist_ok=True)

image_paths = glob(os.path.join(input_folder, "*.jpg"))  

augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),  
    iaa.Flipud(0.3),  
    iaa.Affine(rotate=(-30, 30), scale=(0.8, 1.2)),  
    iaa.Crop(percent=(0, 0.2)), 
    iaa.Grayscale(alpha=(0.0, 1.0)),  
    iaa.AddToHueAndSaturation((-20, 20)),  
    iaa.LinearContrast((0.8, 1.5)), 
    iaa.Affine(rotate=(-20, 20)), 
    iaa.GaussianBlur(sigma=(0, 1.5)), 
     iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
    
])

for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"fail: {img_path}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    for i in range(3):  
        augmented_img = augmenters.augment_image(img)  

        img_name = os.path.basename(img_path)
        save_path = os.path.join(output_folder, f"aug_{i}_" + img_name)
        cv2.imwrite(save_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

print("done:", output_folder)